In [2]:
import scala.io.Source
import java.io.{File, PrintWriter}
import org.apache.spark.sql.{SparkSession, DataFrame, Row}
import org.apache.spark.sql.types.{StructType, StructField, StringType}
import org.apache.spark.sql.functions.{col, split, regexp_replace, when}

import scala.io.Source
import java.io.{File, PrintWriter}
import org.apache.spark.sql.{SparkSession, DataFrame, Row}
import org.apache.spark.sql.types.{StructType, StructField, StringType}
import org.apache.spark.sql.functions.{col, split, regexp_replace, when}


In [3]:
val spark = SparkSession.builder
    .appName("medi_test")
    .getOrCreate()

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@2e38f42


In [4]:
import spark.implicits._

import spark.implicits._


In [6]:
val rootPath = "/Users/b06/Desktop/yeardream/medi-05"
val jsonRootPath = s"$rootPath/data/naverplace_meta"
val saveRootPath = s"$rootPath/spark-scala-project/output/scala"
val textRootPath = s"$rootPath/spark-scala-project/test.txt"

rootPath: String = /Users/b06/Desktop/yeardream/medi-05
jsonRootPath: String = /Users/b06/Desktop/yeardream/medi-05/data/naverplace_meta
saveRootPath: String = /Users/b06/Desktop/yeardream/medi-05/spark-scala-project/output/scala
textRootPath: String = /Users/b06/Desktop/yeardream/medi-05/spark-scala-project/test.txt


In [14]:
def readText(filePath: String): String = {
    val file = Source.fromFile(filePath)
    val lines = file.getLines.toList
    file.close()
    val n = if (lines.nonEmpty) lines.head.trim else ""
    if (lines.nonEmpty) {
        val remainingLines = lines.tail
        val writer = new PrintWriter(new File(filePath))
        remainingLines.foreach(writer.println)
        writer.close()
    }
    n
}

def readJson(n: String, jsonRootPath: String): DataFrame = {
    val jsonPath = s"$jsonRootPath/naverplace_meta_$n.json"
    spark.read.option("multiline", "true").json(jsonPath)
}

readText: (filePath: String)String
readJson: (n: String, jsonRootPath: String)org.apache.spark.sql.DataFrame


In [15]:
val n = readText(textRootPath)
println(n)
val data = readJson(n, jsonRootPath)

1


n: String = 1
data: org.apache.spark.sql.DataFrame = [BaseNaverBlog:betbetter: struct<__typename: string, categoryNo: string ... 1 more field>, BaseNaverBlog:bondiolsc: struct<__typename: string, categoryNo: string ... 1 more field> ... 469 more fields]


In [18]:
val columns = data.columns
val hospitalBases = columns.filter(_.contains("HospitalBase"))

columns: Array[String] = Array(BaseNaverBlog:betbetter, BaseNaverBlog:bondiolsc, BaseNaverBlog:dainhani, BaseNaverBlog:kundaeclinic, BaseNaverBlog:memeetsworld, BaseNaverBlog:orot-hani, BaseNaverBlog:zzz0501, BusStation:104073, BusStation:104091, BusStation:104094, BusStation:104132, BusStation:104140, BusStation:104154, BusStation:104158, BusStation:104172, BusStation:104181, BusStation:104212, BusStation:104222, BusStation:104231, BusStation:104321, BusStation:104386, BusStation:104395, BusStation:104428, BusStation:104440, BusStation:104459, BusStation:104500, BusStation:104507, BusStation:104517, BusStation:104532, BusStation:104544, BusStation:104554, BusStation:104573, BusStation:104576, BusStation:104578, BusStation:104580, BusStation:104582, BusStation:104594, BusStation:104598,...


In [19]:
val dfSchema = StructType(Seq(
    StructField("id", StringType, nullable = true),
    StructField("name", StringType, nullable = true),
    StructField("review_keywords", StringType, nullable = true),
    StructField("description", StringType, nullable = true),
    StructField("road", StringType, nullable = true),
    StructField("booking_business_id", StringType, nullable = true),
    StructField("booking_display_name", StringType, nullable = true),
    StructField("category", StringType, nullable = true),
    StructField("category_code", StringType, nullable = true),
    StructField("category_code_list", StringType, nullable = true),
    StructField("category_count", StringType, nullable = true),
    StructField("rcode", StringType, nullable = true),
    StructField("virtual_phone", StringType, nullable = true),
    StructField("phone", StringType, nullable = true),
    StructField("naver_booking_url", StringType, nullable = true),
    StructField("conveniences", StringType, nullable = true),
    StructField("talktalk_url", StringType, nullable = true),
    StructField("road_address", StringType, nullable = true),
    StructField("keywords", StringType, nullable = true),
    StructField("payment_info", StringType, nullable = true),
    StructField("ref", StringType, nullable = true),
    StructField("lon", StringType, nullable = true),
    StructField("lat", StringType, nullable = true)
))
val df = spark.createDataFrame(spark.sparkContext.emptyRDD[Row], dfSchema)

dfSchema: org.apache.spark.sql.types.StructType = StructType(StructField(id,StringType,true),StructField(name,StringType,true),StructField(review_keywords,StringType,true),StructField(description,StringType,true),StructField(road,StringType,true),StructField(booking_business_id,StringType,true),StructField(booking_display_name,StringType,true),StructField(category,StringType,true),StructField(category_code,StringType,true),StructField(category_code_list,StringType,true),StructField(category_count,StringType,true),StructField(rcode,StringType,true),StructField(virtual_phone,StringType,true),StructField(phone,StringType,true),StructField(naver_booking_url,StringType,true),StructField(conveniences,StringType,true),StructField(talktalk_url,StringType,true),StructField(road_address,StringTyp...


In [46]:
def getValue(data: DataFrame, baseId: String, key: String) = {
  val columnKey = s"HospitalBase:$baseId.$key"
  val column = data.select(columnKey)
  val row = column.filter(!col(key).isNull)
  val value = row.first
  value
}

def replaceExprAndGetValues(value: Option[String]) = {
  value match {
    case Some(v) =>
      val replacedValue = v.replaceAll("\n|\r|,|\\*", "")
      Some(replacedValue)
    case None => None
  }
}

def checkNone(value: Option[String]) = {
  value match {
    case Some(v) => Some(v.charAt(0))
    case None => None
  }
}

def getLonLatValues(refValue: Option[String], data: DataFrame) = {
  refValue match {
    case Some(rv) =>
      val panoramaKey = rv.split(":")(1)
      val panoramaData = data.selectExpr(s"`Panorama:$panoramaKey` as panorama_data")
      val panoramaValue = panoramaData.filter(!col("panorama_data").isNull).first

      panoramaValue match {
        case Some(row) =>
          val lon = row.getMap[String, Double](0)("lon")
          val lat = row.getMap[String, Double](0)("lat")
          Some(Map("lon" -> lon, "lat" -> lat))
        case None => None
      }
    case None => None
  }
}

def createBooleanColumn(df: DataFrame, targetColumn: String, createColumn: String, value: Option[String] = None) = {
  val condition = value match {
    case Some(v) => col(targetColumn).isNotNull && col(targetColumn).contains(v)
    case None    => col(targetColumn).isNotNull && col(targetColumn) =!= ""
  }
  df.withColumn(createColumn, when(condition, true).otherwise(false))
}

def expandColumn(df: DataFrame, column: String, maxNum: Int) = {
  val createArray = s"${column}_array"
  var newDf = df.withColumn(createArray, split(col(column), ","))
  for (i <- 0 until maxNum) {
    newDf = newDf.withColumn(s"${column}_${i + 1}", col(createArray)(i))
  }
  newDf
}

def replaceExprInKeywordColumns(df: DataFrame) = {
  var newDf = df
  for (i <- 1 to 5) {
    val keywordColumn = s"keywords_$i"
    newDf = newDf.withColumn(
      keywordColumn,
      when(col(keywordColumn).isNotNull, regexp_replace(col(keywordColumn), "[\\[\\]]", ""))
    )
  }
  newDf
}

<console>: 71: error: constructor cannot be instantiated to expected type;

In [45]:
var hospitalData = Seq[Row]()
for (hospitalBase <- hospitalBases) {
    val baseId = hospitalBase.split(":")(1).trim
    val idValue = getValue(data, baseId, "id")
    val nameValue = getValue(data, baseId, "name")
    val reviewKeywordsValue = getValue(data, baseId, "reviewSettings").map(_.getAs[String](0))
    val descriptionValue = getValue(data, baseId, "description").map(_.getAs[String](0))
    val roadValue = getValue(data, baseId, "road").map(_.getAs[String](0))
    val bookingBusinessIdValue = getValue(data, baseId, "bookingBusinessId").map(_.getAs[String](0))
    val bookingDisplayNameValue = getValue(data, baseId, "bookingDisplayName").map(_.getAs[String](0))
    val categoryValue = getValue(data, baseId, "category").map(_.getAs[String](0))
    val categoryCodeValue = getValue(data, baseId, "categoryCode").map(_.getAs[String](0))
    val categoryCodeListValue = getValue(data, baseId, "categoryCodeList").map(_.getAs[String](0))
    val categoryCountValue = getValue(data, baseId, "categoryCount").map(_.getAs[String](0))
    val rcodeValue = getValue(data, baseId, "rcode").map(_.getAs[String](0))
    val virtualPhoneValue = getValue(data, baseId, "virtualPhone").map(_.getAs[String](0))
    val phoneValue = getValue(data, baseId, "phone").map(_.getAs[String](0))
    val naverBookingUrlValue = getValue(data, baseId, "naverBookingUrl").map(_.getAs[String](0))
    val conveniencesValue = getValue(data, baseId, "conveniences").map(_.getAs[String](0))
    val talktalkUrlValue = getValue(data, baseId, "talktalkUrl").map(_.getAs[String](0))
    val roadAddressValue = getValue(data, baseId, "roadAddress").map(_.getAs[String](0))
    val keywordsValue = getValue(data, baseId, "keywords").map(_.getAs[String](0))
    val paymentInfoValue = getValue(data, baseId, "paymentInfo").map(_.getAs[String](0))
    val streetPanoramaValue = getValue(data, baseId, "streetPanorama")
    val refValue = streetPanoramaValue.flatMap(spv => Option(spv.getAs[String]("__ref")))
    val lonLatValues = refValue.flatMap { rv =>
        print(rv)
        getLonLatValues(rv, data)
    }.getOrElse((null, null))

    val roadValueReplaced = roadValue.map(_.replaceAll("\n|\r|,|\\*", ""))
    val descriptionValueReplaced = descriptionValue.map(_.replaceAll("\n|\r|,|\\*", ""))
    val reviewKeywordsValueReplaced = reviewKeywordsValue.map(_.replaceAll("\\\\", "").replaceAll("\"", ""))
    println(s"replaced HospitalBase:$baseId's expressions")

    val row = Row(
        baseId,
        nameValue.getOrElse(null),
        reviewKeywordsValueReplaced.getOrElse(null),
        descriptionValueReplaced.getOrElse(null),
        roadValueReplaced.getOrElse(null),
        bookingBusinessIdValue.getOrElse(null),
        bookingDisplayNameValue.getOrElse(null),
        categoryValue.getOrElse(null),
        categoryCodeValue.getOrElse(null),
        categoryCodeListValue.getOrElse(null),
        categoryCountValue.getOrElse(null),
        rcodeValue.getOrElse(null),
        virtualPhoneValue.getOrElse(null),
        phoneValue.getOrElse(null),
        naverBookingUrlValue.getOrElse(null),
        conveniencesValue.getOrElse(null),
        talktalkUrlValue.getOrElse(null),
        roadAddressValue.getOrElse(null),
        keywordsValue.getOrElse(null),
        paymentInfoValue.getOrElse(null),
        refValue.getOrElse(null),
        lonLatValues._1,
        lonLatValues._2
    )
    hospitalData = hospitalData :+ row
    println(s"appended HospitalBase:$baseId's rows")
}

<console>: 70: error: type mismatch;